## Gradient Descent and Linear Regression with PyTorch

### Part 2 of "Deep Learning with Pytorch: Zero to GANs"

This tutorial series is a hands-on beginner-friendly introduction to deep learning using [PyTorch](https://pytorch.org), an open-source neural networks library. These tutorials take a practical and coding-focused approach. The best way to learn the material is to execute the code and experiment with it yourself. Check out the full series here:

1. [PyTorch Basics: Tensors & Gradients](https://jovian.ai/aakashns/01-pytorch-basics)
2. [Gradient Descent & Linear Regression](https://jovian.ai/aakashns/02-linear-regression)
3. [Working with Images & Logistic Regression](https://jovian.ai/aakashns/03-logistic-regression) 
4. [Training Deep Neural Networks on a GPU](https://jovian.ai/aakashns/04-feedforward-nn)
5. [Image Classification using Convolutional Neural Networks](https://jovian.ai/aakashns/05-cifar10-cnn)
6. [Data Augmentation, Regularization and ResNets](https://jovian.ai/aakashns/05b-cifar10-resnet)
7. [Generating Images using Generative Adversarial Networks](https://jovian.ai/aakashns/06b-anime-dcgan/)


## Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

In [2736]:
# linear regressoin is one of the fundamental algorithms of machine learning
# you basically multiply the independent variables by a weight and then add a bias offset, basically you test an initial weight and then 

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` using the training data, to make accurate predictions for new data. The _learned_ weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region. 

We'll _train_ our model by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*. Let's begin by importing Numpy and PyTorch.

In [2737]:
import numpy as np
import torch
#necessary inputs

## Training data

We can represent the training data using two matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [2738]:
# Input (temp, rainfall, humidity)
#this is the inputs array (just a numpy array)
#first kanto, second johto, third hoenn, fourth sinnoh, fifth unova
# the first category is temp (F), second rainfall mm, third humidity 
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')


In [2739]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

We've separated the input and target variables because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors.

Let's convert the arrays to PyTorch tensors.

In [2740]:
# Convert inputs and targets to tensors
#basically the process is use pandas to load in your inputs and targets then convert to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [2741]:
# Weights and biases
# torch rand picks randomly from a normal distribution with mean 0 and standard deviation 1
# there are two different targets each requiring 3 different weights, so the weights matrix has to be 2x3
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.0522, -1.1931,  0.3418],
        [ 0.5824,  0.0763, -1.6385]], requires_grad=True)
tensor([ 2.2341, -1.7922], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [2742]:
#the weights have to be transposed  to conform to the matrix multiplicaiton 
#since the inputs matrix is 5x3 the weights has to be 3x2 and the bias fector doesn't matter
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [2743]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 13.8037, -24.6223],
        [ 14.8656, -46.9458],
        [-46.2768, -35.9337],
        [ 70.9017,   0.2657],
        [-15.7772, -68.9783]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [2744]:
# Compare with targets
print(targets)
#obviously vastly different from the targets because we randomly generated the whatever

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [2745]:
# MSE loss
#simple mean squared error
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()
#.numel returns the number of elements in the tensor
#this tells us each element in the predictuoin differes from the actual target by the square root of the loss

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [2746]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(14603.5264, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by the square root of the loss*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the *loss* because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [2747]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [2748]:
# Gradients for weights
print("gradient for weight")
print(w)
print(w.grad)

print("gradient for biases")
print(b)
print(b.grad)

gradient for weight
tensor([[ 1.0522, -1.1931,  0.3418],
        [ 0.5824,  0.0763, -1.6385]], requires_grad=True)
tensor([[ -5337.0591,  -8018.7812,  -4427.6274],
        [-10357.0254, -12324.3057,  -7570.6191]])
gradient for biases
tensor([ 2.2341, -1.7922], requires_grad=True)
tensor([ -68.6966, -127.2429])


## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [2749]:
w
w.grad

tensor([[ -5337.0591,  -8018.7812,  -4427.6274],
        [-10357.0254, -12324.3057,  -7570.6191]])

In [2750]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [2751]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(14603.5264, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [2752]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [2753]:
# Generate predictions
#process is always generate pred, calculate loss, backwards deriv, subtract small gradiant, reset grad to zero.
preds = model(inputs)
print(preds)

tensor([[ 24.9769,  -5.5477],
        [ 29.6132, -21.8290],
        [-28.3197,  -6.0162],
        [ 81.4324,  18.9318],
        [ -1.2966, -44.6999]], grad_fn=<AddBackward0>)


In [2754]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(10696.0195, grad_fn=<DivBackward0>)


In [2755]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -4178.4097,  -6759.6587,  -3653.8093],
        [ -8385.0195, -10198.1924,  -6260.0151]])
tensor([ -54.9187, -103.8322])


Let's update the weights and biases using the gradients computed above.

In [2756]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

Let's take a look at the new weights and biases.

In [2757]:
print(w)
print(b)

tensor([[ 1.1474, -1.0453,  0.4226],
        [ 0.7698,  0.3015, -1.5002]], requires_grad=True)
tensor([ 2.2353, -1.7899], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [2758]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(8052.8438, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss merely by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [2759]:
# Train for 100 epochs
# 1 e-5 is a hyperparameter, see what works 
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [2760]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)
print("average loss: " + str(np.sqrt(loss.detach().numpy())))

tensor(876.0812, grad_fn=<DivBackward0>)
average loss: 29.59867


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [2761]:
# Predictions
preds

tensor([[ 67.6185,  77.1261],
        [ 90.5135,  91.0607],
        [ 83.0031, 143.8485],
        [ 79.7930,  76.9425],
        [ 82.5494,  78.7680]], grad_fn=<AddBackward0>)

In [2762]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs. 

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [2763]:
import torch.nn as nn
# we don't have to build it manually and we can train using built in libraries

As before, we represent the inputs and targets and matrices.

In [2764]:
# Input (temp, rainfall, humidity)
# we're working with 15 different datasets instead of 5
# ususally we only train off a small part of the data set and we use validation to verify it
# using smaller matricies is better becasue large matrix multiplicaiton is very costly
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [2765]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

We are using 15 training examples to illustrate how to work with large datasets in small batches. 

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [2766]:
from torch.utils.data import TensorDataset
#we need to break the dataset into smaller parts

In [2767]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

#gives you first three rows of inputs
#then first three rrows of targers

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [2768]:
from torch.utils.data import DataLoader

In [2769]:
# Define data loader
# pass in training detaset and batches of dataset
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a `for` loop. Let's look at an example.

In [2770]:
for xb, yb in train_dl:
    print('batch:')
    print(xb)
    print(yb)
    break

batch:
tensor([[ 88., 134.,  59.],
        [ 91.,  87.,  65.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 73.,  66.,  44.]])
tensor([[118., 132.],
        [ 80., 102.],
        [ 22.,  37.],
        [ 56.,  70.],
        [ 57.,  69.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [2771]:
# Define model
# nn. linear does a basic linear regression without us having to define it manually
# bias is a vector of size 2
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1873,  0.3503, -0.3835],
        [-0.4803,  0.4555, -0.3416]], requires_grad=True)
Parameter containing:
tensor([-0.1862,  0.0044], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [2772]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.1873,  0.3503, -0.3835],
         [-0.4803,  0.4555, -0.3416]], requires_grad=True),
 Parameter containing:
 tensor([-0.1862,  0.0044], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [2773]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -6.8824, -19.2290],
        [-10.9523, -25.4830],
        [  8.2098,  -0.5606],
        [-18.4182, -42.0387],
        [ -6.3315, -13.3228],
        [ -7.4199, -20.1647],
        [-11.6861, -26.2801],
        [  7.6390,  -1.3825],
        [-17.8807, -41.1030],
        [ -6.5277, -13.1841],
        [ -7.6162, -20.0261],
        [-11.4898, -26.4188],
        [  8.9436,   0.2365],
        [-18.2220, -42.1774],
        [ -5.7939, -12.3870]], grad_fn=<AddmmBackward0>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [2774]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [2775]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [2776]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(10386.5693, grad_fn=<MseLossBackward0>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [2777]:
# Define optimizer
#use a stochastic gradient descent to optimize the model
# model.parameters are the things that need to be updated
# has the gradient as a pert of the model
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [2778]:
# Utility function to train the model
# this allows us to modify easily, we just put in everything that we have and we let this do the work
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # batches in dataset 
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients, updates them from zero
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        #loss is a tensor and the item gets the numerical value

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [2779]:
fit(500, model, loss_fn, opt, train_dl)

Epoch [10/500], Loss: 306.2371
Epoch [20/500], Loss: 137.7837
Epoch [30/500], Loss: 99.0105
Epoch [40/500], Loss: 157.0631
Epoch [50/500], Loss: 116.6572
Epoch [60/500], Loss: 130.7298
Epoch [70/500], Loss: 85.4811
Epoch [80/500], Loss: 51.5243
Epoch [90/500], Loss: 82.4509


Epoch [100/500], Loss: 43.0389
Epoch [110/500], Loss: 47.8127
Epoch [120/500], Loss: 47.9212
Epoch [130/500], Loss: 41.0461
Epoch [140/500], Loss: 36.4354
Epoch [150/500], Loss: 45.0747
Epoch [160/500], Loss: 44.7640
Epoch [170/500], Loss: 48.5533
Epoch [180/500], Loss: 24.3748
Epoch [190/500], Loss: 40.6967
Epoch [200/500], Loss: 7.8028
Epoch [210/500], Loss: 30.8908
Epoch [220/500], Loss: 33.1213
Epoch [230/500], Loss: 24.4376
Epoch [240/500], Loss: 27.3861
Epoch [250/500], Loss: 26.9544
Epoch [260/500], Loss: 19.4002
Epoch [270/500], Loss: 19.6921
Epoch [280/500], Loss: 25.0680
Epoch [290/500], Loss: 14.3282
Epoch [300/500], Loss: 15.4479
Epoch [310/500], Loss: 25.8788
Epoch [320/500], Loss: 11.1351
Epoch [330/500], Loss: 9.1044
Epoch [340/500], Loss: 7.5806
Epoch [350/500], Loss: 9.9922
Epoch [360/500], Loss: 9.1836
Epoch [370/500], Loss: 4.0150
Epoch [380/500], Loss: 10.7038
Epoch [390/500], Loss: 7.3144
Epoch [400/500], Loss: 10.0724
Epoch [410/500], Loss: 9.6074
Epoch [420/500],

Let's generate predictions using our model and verify that they're close to our targets.

In [2780]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.1366,  70.6263],
        [ 80.8518,  99.1730],
        [121.0077, 135.9289],
        [ 21.6026,  38.6149],
        [ 99.2819, 115.5967],
        [ 55.8638,  69.5167],
        [ 80.4706,  99.0006],
        [121.1748, 136.3849],
        [ 22.8754,  39.7245],
        [100.1735, 116.5338],
        [ 56.7554,  70.4539],
        [ 79.5790,  98.0634],
        [121.3889, 136.1013],
        [ 20.7110,  37.6778],
        [100.5547, 116.7062]], grad_fn=<AddmmBackward0>)

In [2781]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [2782]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.2994, 67.3658]], grad_fn=<AddmmBackward0>)

The predicted yield of apples is 54.3 tons per hectare, and that of oranges is 68.3 tons per hectare.

## Machine Learning vs. Classical Programming

The approach we've taken in this tutorial is very different from programming as you might know it. Usually, we write programs that take some inputs, perform some operations, and return a result. 

However, in this notebook, we've defined a "model" that assumes a specific relationship between the inputs and the outputs, expressed using some unknown parameters (weights & biases). We then show the model some know inputs and outputs and _train_ the model to come up with good values for the unknown parameters. Once trained, the model can be used to compute the outputs for new inputs.

This paradigm of programming is known as _machine learning_, where we use data to figure out the relationship between inputs and outputs. _Deep learning_ is a branch of machine learning that uses matrix operations, non-linear activation functions and gradient descent to build and train models. Andrej Karpathy, the director of AI at Tesla Motors, has written a great blog post on this topics, titled [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35).

This picture from book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by Francois Chollet captures the difference between classical programming and machine learning:

![](https://i.imgur.com/oJEQe7k.png)

Keep this picture in mind as you work through the next few tutorials. 



## Commit and update the notebook

As a final step, we can record a new version of the notebook using the `jovian` library.

Note that running `jovian.commit` a second time records a new version of your existing notebook. With Jovian.ml, you can avoid creating copies of your Jupyter notebooks and keep versions organized. Jovian also provides a visual diff ([example](https://jovian.ai/aakashns/keras-mnist-jovian/diff?base=8&remote=2)) so you can inspect what has changed between different versions:

![jovian-commenting](https://i.imgur.com/HF1cOVt.png)

## Exercises and Further Reading

We've covered the following topics in this tutorial:

- Introduction to linear regression and gradient descent
- Implementing a linear regression model using PyTorch tensors
- Training a linear regression model using the gradient descent algorithm
- Implementing gradient descent and linear regression using PyTorch built-in


Here are some resources for learning more about linear regression and gradient descent:

* An visual & animated explanation of gradient descent: https://www.youtube.com/watch?v=IHZwWFHWa-w

* For a more detailed explanation of derivates and gradient descent, see [these notes from a Udacity course](https://storage.googleapis.com/supplemental_media/udacityu/315142919/Gradient%20Descent.pdf). 

* For an animated visualization of how linear regression works, [see this post](https://hackernoon.com/visualizing-linear-regression-with-pytorch-9261f49edb09).

* For a more mathematical treatment of matrix calculus, linear regression and gradient descent, you should check out [Andrew Ng's excellent course notes](https://github.com/Cleo-Stanford-CS/CS229_Notes/blob/master/lectures/cs229-notes1.pdf) from CS229 at Stanford University.

* To practice and test your skills, you can participate in the [Boston Housing Price Prediction](https://www.kaggle.com/c/boston-housing) competition on Kaggle, a website that hosts data science competitions.

With this, we complete our discussion of linear regression in PyTorch, and we’re ready to move on to the next topic: [Working with Images & Logistic Regression](https://jovian.ai/aakashns/03-logistic-regression).

## Questions for Review

Try answering the following questions to test your understanding of the topics covered in this notebook:

1. What is a linear regression model? Give an example of a problem formulated as a linear regression model.

A linear regression model basically tries to map a set of numbers to a linear curve, an example is just having numbers for temperature and rainfall and having a yield
It's commonly used if you have inputs and targets that are numerical values.

2. What are input and target variables in a dataset? Give an example.

inputs are the inputs you have like the data over an environment
targets are the things you're trying to achieve


3. What are weights and biases in a linear regression model?

basically you can express the target output as a linear combination of the inputs multiplied by a weights matrix and addint an offset bias vector.

4. How do you represent tabular data using PyTorch tensors?

you load them into a numpy array and then convert that into a tensor

5. Why do we create separate matrices for inputs and targets while training a linear regression model?

that's kind of intuitive, inputs are inputs and the targets are what you're tyring to acheive
in machine learning you're letting the model determine the rules to figure out the targets

6. How do you determine the shape of the weights matrix & bias vector given some training data?

the bias vector is as big as the number of targets you may have
like in our equation there are two targets, meaning that there are two linear equations

the weights matrix has the number of rows (= number of targets) by the number of inputs 
so if you have two targets and 3 inputs the weights matrix is 2x3 

however we need to transpose the matrix to you know make the multiplicaiotn possible because 

mxn mxn where the internam n and m have to be the same for matrix multiplication to occur


7. How do you create randomly initialized weights & biases with a given shape?

uhhh you use a random generator 
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

or you can use model = nn.Linear while including the parameters to fill it

8. How is a linear regression model implemented using matrix operations? Explain with an example.

Ax + b = target 

more specifically 
Inputs w^T + b = target vector

then we do a little multivariate calculus

9. How do you generate predictions using a linear regression model?

lol you just put some inputs into the matrix equation and then you're golden.

10. Why are the predictions of a randomly initialized model different from the actual targets?

because they are randomly generated weights, we're trying to train a model to descend and figure out the weights and biases that are correct

you're just plugging in random weights and biases i mean it could work but it's not going to work

11. What is a loss function? What does the term “loss” signify?

a loss function basically helps determine how different the values are from what they're supposed to be 

12. What is mean squared error?

a mean squared error is you sum all the differences squared and divide them by the number of objects
the reason why it's often squared is that you don't have negative numbers in your loss

13. Write a function to calculate mean squared using model predictions and actual targets.

def mse(t1,t2):
    diff = t1-t2
    return torch.sum(diff * diff) / diff.numel()



14. What happens when you invoke the `.backward` function on the result of the mean squared error loss function?

takes the backwards derivatives of the loss function wrt to w and b because we set requires_grad equal to true for those values
so that the derivatves thave 

15. Why is the derivative of the loss w.r.t. the weights matrix itself a matrix? What do its elements represent?

its a matrix because the weights are a matrix. each element changes wrt to the loss fucntion differently and they need to be adjusted in different ways

16. How is the derivate of the loss w.r.t. a weight element useful for reducing the loss? Explain with an example.

basically it tells you if increasing or decreasing the value of the weights increases or decreses the loss. then you can adjust accordingly to decrease the loss

17. Suppose the derivative  of the loss w.r.t. a weight element is positive. Should you increase or decrease the element’s value slightly to get a lower loss?

if the derivative of the loss w.r.t a weight element is positive the element's value then you should decrease the element's value to get a lower loss

18. Suppose the derivative  of the loss w.r.t. a weight element is negative. Should you increase or decrease the element’s value slightly to get a lower loss?

if the derivative of the loss w.r.t a weight element is negative, then you should increase the element's value to get a lower loss.

19. How do you update the weights and biases of a model using their respective gradients to reduce the loss slightly?

you just subtract a 1e-5 * w.grad example becuase the gradients can be big or small so just adjust in very small steps
works for the negative case because subtracting a negative actually adds the number as an adjusstment.

20. What is the gradient descent optimization algorithm? Why is it called “gradient descent”?

using the gradient to adjust the loss and optimizaiton. it's called the gradient descent because you're taking the gradient until you get a plateau or saddle point to achieve optimization.
you're adjusting it until the steepest ascent is zero.

21. Why do you subtract a “small quantity” proportional to the gradient from the weights & biases, not the actual gradient itself?

because the gradient is the point of steepest ascent, the rate at which something is changinging, you don't want to make a large cut and end up in a tick tock clock corrupted pendulum motion.
do not want to take a step that is too big

22. What is learning rate? Why is it important?

learning rate is the rate at which it's actually improving, you don't want an algorithm that is incredibly slow at learning with bonkers matrix multiplication.

23. What is `torch.no_grad`?

indicates that the gradient is not being calcualted at the time and temporarily sets all the requires_grad flages to false , saves memeory nad computation 
used for adjusting due to the gradients of the loss wrt of w and b.

24. Why do you reset gradients to zero after updating weights and biases?

because if you don't the new gradient calculation adds to the gradient vector and that is very bad so you reset it to zero every time.

25. What are the steps involved in training a linear regression model using gradient descent?

a. generate random weights and biases. 
b. define loss function
c. calculate backwards derivatives of loss funciotn with respect to weights and biases
d. adjust weights and biases with a small increment then reset each gradient back to zero.
e. repeat

26. What is an epoch?

the iteration counter of how many times you're optimizing the model using a backwards gradient descent

27. What is the benefit of training a model for multiple epochs?

the model gets more optimized per each iteration

28. How do you make predictions using a trained model?

same way you make predictions using an untrained model, you just plug inputs into the model

29. What should you do if your model’s loss doesn’t decrease while training? Hint: learning rate.

you're either taking too large of seteps per epoch or you've done so many epochs that you're plateauing

30. What is `torch.nn`?

its the PyTorch neural netork library

31. What is the purpose of the `TensorDataset` class in PyTorch? Give an example.

the TensorDataset class in PyTorch is used to segmenet our dataset into smaller parts

32. What is a data loader in PyTorch? Give an example.

a dataloader can load in specific bastches of data

33. How do you use a data loader to retrieve batches of data?

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

you ust use the training dataset with a batch side and suffle

34. What are the benefits of shuffling the training data before creating batches?

don't have adjacent and similar values next to each other like if you have something alphabetized in the dataset it's best to train off
of random packets to get the quickest results

35. What is the benefit of training in small batches instead of training with the entire dataset?

can only have so much stored in memory at the same time.

36. What is the purpose of the `nn.Linear` class in PyTorch? Give an example.

the nn.linear class is a basic linear regression model.

37. How do you see the weights and biases of a `nn.Linear` model?

you callmodel = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

or you do list(model.parameters()) or print that 

38. What is the purpose of the `torch.nn.functional` module?

torch.nn.functional module has loss fucntions 

39. How do you compute mean squared error loss using a PyTorch built-in function?

you import torch.nn.functional as Floss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)

40. What is an optimizer in PyTorch?

an optimizer takes the loss and minimizes the orginal factors to optimize the weights and biases

41. What is `torch.optim.SGD`? What does SGD stand for?

stochastic gradient descent
adjusts based on a gradient descent algorithm

42. What are the inputs to a PyTorch optimizer? 

you just need the model's parameters and then the factor of adjustment

43. Give an example of creating an optimizer for training a linear regression model.

opt = torch.optim.SGD(model.parameters(), lr=1e-5)
            # 3. Compute gradients, updates them from zero
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()

44. Write a function to train a `nn.Linear` model in batches using gradient descent.

no

45. How do you use a linear regression model to make predictions on previously unseen data?

you plug in the unseen data as the inputs and then compare the model's output to the original data expected targets